In [ ]:
# This cell will be deleted as its content is being split into new, more organized cells.

In [ ]:
# --- 1. INSTALAÇÃO E IMPORTAÇÕES ESSENCIAIS ---
# Este bloco garante que a biblioteca LTNtorch esteja instalada na versão correta
# e importa todas as bibliotecas Python necessárias para o projeto.

# Remove qualquer versão existente do LTN para evitar conflitos.
!pip uninstall ltn -y
# Instala a versão específica do LTNtorch diretamente do repositório GitHub,
# garantindo compatibilidade e acesso às funcionalidades mais recentes.
!pip install git+https://github.com/logictensornetworks/LTNtorch

# Importa a biblioteca LTNtorch, fundamental para a construção de redes tensor lógicas.
import ltn
# Importa a biblioteca PyTorch, a base para operações de tensores e redes neurais.
import torch
# Importa o módulo de redes neurais do PyTorch, usado para definir os modelos.
import torch.nn as nn
# Importa NumPy para operações numéricas, especialmente útil para manipulação de arrays.
import numpy as np
# Importa métricas do scikit-learn para avaliação de modelos, como acurácia e F1-score.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- CONFIGURAÇÃO GLOBAL DO DISPOSITIVO ---
# Configura o dispositivo (GPU CUDA ou CPU) a ser usado para todas as operações de tensores.
# Prioriza a GPU se disponível, caso contrário, utiliza a CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Rodando em: {device}")

print("Configuração inicial e importações concluídas.")

  Cloning https://github.com/logictensornetworks/LTNtorch to /tmp/pip-req-build-0j68s36_
  Running command git clone --filter=blob:none --quiet https://github.com/logictensornetworks/LTNtorch /tmp/pip-req-build-0j68s36_
  Resolved https://github.com/logictensornetworks/LTNtorch to commit d1bd98169cc2121f8cdd25ff99901e4589923c95
  Preparing metadata (setup.py) ... done
  Created wheel for LTNtorch: filename=LTNtorch-1.0.2-py3-none-any.whl size=29525 sha256=8afc9e0cdb1a729400162993f01a27cafb20ea53cbce94aa0cf1d91168ebb07f
  Stored in directory: /tmp/pip-ephem-wheel-cache-evxrwbvh/wheels/da/f9/39/427ffc120e0d781e4de19bbed6a73c2e766ba29f0e5d7fb589
Successfully built LTNtorch
Rodando em: cpu
Configuração inicial e importações concluídas.


In [ ]:
# --- 2. GERAÇÃO DE DADOS E DEFINIÇÃO DOS MODELOS NEURAIS ---
# Este bloco contém a lógica para gerar dados sintéticos (ambiente ClevrSimplified)
# e as definições das classes de modelos neurais que servem como predicados no LTN.

# --- GERAÇÃO DE DADOS (CLASSE CLEVR SIMPLIFIED) ---
# A classe ClevrSimplified gera um conjunto de dados sintéticos que simulam objetos
# com diversas propriedades (posição, cor, forma, tamanho). Cada objeto é um vetor.
class ClevrSimplified:
    def __init__(self, num_samples=50):
        self.num_samples = num_samples
        self.data = self._generate()

    # Método privado para gerar os vetores de características para cada objeto.
    # Cada vetor contém: [pos_x, pos_y, color_r, color_g, color_b, shape_circ, shape_sq, shape_cyl, shape_cone, shape_tri, size]
    def _generate(self):
        data = []
        for _ in range(self.num_samples):
            pos = np.random.rand(2) # Posição (x, y) normalizada entre 0 e 1
            c_idx = np.random.randint(0, 3) # Índice da cor (0:R, 1:G, 2:B)
            color = np.zeros(3); color[c_idx] = 1 # Representação one-hot da cor
            s_idx = np.random.randint(0, 5) # Índice da forma (0:Circle, 1:Square, etc.)
            shape = np.zeros(5); shape[s_idx] = 1 # Representação one-hot da forma
            is_large = np.random.rand() > 0.5 # Booleano para definir se o tamanho é grande
            size = np.array([1.0 if is_large else 0.0]) # Tamanho (1.0 para grande, 0.0 para pequeno)
            vec = np.concatenate([pos, color, shape, size]) # Concatena todas as características em um único vetor
            data.append(vec)
        # Converte a lista de vetores para um tensor PyTorch e move para o dispositivo configurado.
        return torch.tensor(np.array(data), dtype=torch.float32).to(device)

# --- DEFINIÇÃO DOS MODELOS NEURAIS PARA PREDICADOS LTN ---
# Estes modelos são usados dentro dos predicados LTN para inferir a verdade
# de proposições (e.g., 'É um círculo', 'Está à esquerda de').

# FeatureModel: Modelo neural genérico para predicados unários (propriedades de um único objeto).
# Recebe um subconjunto das características do objeto (definido por input_indices).
class FeatureModel(nn.Module):
    def __init__(self, input_indices):
        super(FeatureModel, self).__init__()
        self.indices = input_indices # Índices das características a serem usadas (e.g., [5] para IsCircle)
        input_dim = len(input_indices) # Dimensão da entrada do modelo
        self.net = nn.Sequential( # Rede neural simples com uma camada oculta e ELU/Sigmoid.
            nn.Linear(input_dim, 16), nn.ELU(), nn.Linear(16, 1), nn.Sigmoid()
        )
    def forward(self, x):
        # Seleciona as características relevantes e passa pela rede neural.
        return self.net(x[..., self.indices])

# RelationModel: Modelo neural genérico para predicados binários (relações entre dois objetos).
# Recebe características de dois objetos (e.g., posições para LeftOf).
class RelationModel(nn.Module):
    def __init__(self, axis_idx):
        super(RelationModel, self).__init__()
        self.axis = axis_idx # Eixo a ser considerado (0 para X, 1 para Y)
        self.net = nn.Sequential( # Rede neural para combinar as características dos dois objetos.
            nn.Linear(2, 16), nn.ELU(), nn.Linear(16, 1), nn.Sigmoid()
        )
    def forward(self, x, y):
        # Concatena a característica do eixo de ambos os objetos e passa pela rede.
        return self.net(torch.cat([x[..., self.axis:self.axis+1], y[..., self.axis:self.axis+1]], dim=-1))

# CloseToModel: Predicado específico para verificar a proximidade entre dois objetos.
# Calcula a distância euclidiana entre as posições (primeiros 2 índices) dos objetos.
class CloseToModel(nn.Module):
    def forward(self, x, y):
        # Calcula o quadrado da distância entre os vetores de posição (0:2).
        dist_sq = torch.sum((x[..., 0:2] - y[..., 0:2])**2, dim=-1, keepdim=True)
        # Retorna um valor baseado na exponencial negativa da distância (quanto menor a distância, mais próximo de 1).
        return torch.exp(-2.0 * dist_sq)

# SameSizeModel: Predicado específico para verificar se dois objetos têm o mesmo tamanho.
# Compara o valor da característica de tamanho (índice 10) dos objetos.
class SameSizeModel(nn.Module):
    def forward(self, x, y):
        # Retorna 1 menos o valor absoluto da diferença de tamanho (1 se iguais, 0 se muito diferentes).
        return 1.0 - torch.abs(x[..., 10:11] - y[..., 10:11])

print("Modelos neurais e classe de geração de dados definidos.")

Modelos neurais e classe de geração de dados definidos.


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Caminho da imagem (atualizado para a imagem do usuário)
# `image_path` é atualizado na célula 00b9437a

# Define as transformações de pré-processamento para a imagem:
# 1. Redimensiona a imagem para (224, 224) pixels - tamanho comum para modelos de visão.
# 2. Converte a imagem PIL para um tensor PyTorch.
# 3. Normaliza o tensor com média e desvio padrão padrão para modelos pré-treinados em ImageNet.
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

try:
    # Carregar a imagem do caminho especificado e converter para o formato RGB.
    original_image = Image.open(image_path).convert("RGB") # Usar 'original_image' para consistência.
    print(f"Imagem carregada do caminho: {image_path}")

    # Aplica as transformações definidas à imagem.
    preprocessed_image = preprocess(original_image)

    # Adiciona uma dimensão de batch ao tensor pré-processado (modelos de rede neural geralmente esperam um batch de imagens).
    preprocessed_image = preprocessed_image.unsqueeze(0)

    print(f"Imagem pré-processada com sucesso. Formato final: {preprocessed_image.shape}")
    print(f"Tipo de dado: {preprocessed_image.dtype}")

except FileNotFoundError:
    print(f"Erro: Imagem não encontrada no caminho: {image_path}")
    original_image = None # Define como None em caso de erro
    preprocessed_image = None
except Exception as e:
    print(f"Ocorreu um erro ao carregar ou pré-processar a imagem: {e}")
    original_image = None # Define como None em caso de erro
    preprocessed_image = None

Ocorreu um erro ao carregar ou pré-processar a imagem: name 'image_path' is not defined


In [ ]:
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# Garante que ultralytics esteja instalado para YOLOv5
# A instalação é importante para que o torch.hub.load funcione corretamente.
!pip install ultralytics

# Garante que o dispositivo está disponível (definido no notebook principal)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# `device` já foi definido na célula 1 e está disponível globalmente.

print(f"Carregando modelo YOLOv5 no dispositivo: {device}")
# 1. Carregar um modelo pré-treinado de detecção de objetos (YOLOv5).
# Usamos `torch.hub.load` que gerencia o download e carregamento do modelo 'yolov5s' (versão small).
# `pretrained=True` garante que o modelo venha com pesos pré-treinados no dataset COCO.
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)
model.eval() # Definir o modelo para modo de avaliação para desativar o dropout e normalização em batch.

print("Modelo YOLOv5 carregado com sucesso.")

# A imagem original `original_image` é carregada na célula `new_cell_5`.
# Garantir que `original_image` esteja disponível
if original_image is None:
    print("Erro: A imagem original não foi carregada. Por favor, execute new_cell_5 primeiro.")
    # Pode ser necessário adicionar uma lógica de fallback ou parar aqui
    # Para evitar que o restante da célula falhe, vamos usar um objeto dummy se original_image não estiver disponível.
    # No entanto, a instrução para reexecutar new_cell_5 deve resolver isso.
    detected_objects_features = torch.tensor([[0.5, 0.5, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5]], dtype=torch.float32).to(device)
    print("Nenhum objeto detectado, usando objeto dummy como fallback.")
else:
    print("Realizando inferência do modelo...")
    # 2. Passar a imagem original pelo modelo de detecção de objetos.
    # O `size=224` redimensiona a imagem internamente para a inferência.
    results = model(original_image, size=224)

    # Os resultados são uma lista de objetos 'Detection'. `results.pred` contém as detecções
    # formatadas como [xmin, ymin, xmax, ymax, confidence, class_id].
    detections = results.pred[0] # Pega as detecções para a primeira (e única) imagem do batch.

    print(f"Número total de objetos detectados (antes do filtro): {len(detections)}")

    # Definir um threshold de confiança para filtrar detecções fracas.
    # `confidence_threshold` é uma variável do kernel com valor 0.5 (definido em uma célula anterior).
    filtered_detections = detections[detections[:, 4] > confidence_threshold]

    print(f"Número de objetos detectados após filtro de confiança ({confidence_threshold}): {len(filtered_detections)}")

    # --- EXTRAÇÃO E NORMALIZAÇÃO DE CARACTERÍSTICAS PARA O FORMATO LTN ---
    # O formato LTN esperado é: [pos_x, pos_y, color_r, color_g, color_b, shape_circ, shape_sq, shape_cyl, shape_cone, shape_tri, size]

    # Se não houver detecções com confiança suficiente, cria um objeto dummy para evitar erros.
    if len(filtered_detections) == 0:
        print("Nenhum objeto detectado com confiança suficiente. Criando um objeto dummy.")
        # Objeto dummy: centro, verde, quadrado, tamanho médio.
        # pos_x=0.5, pos_y=0.5, color_r=0, color_g=1, color_b=0, shape_circ=0, shape_sq=1, shape_cyl=0, shape_cone=0, shape_tri=0, size=0.5
        detected_objects_features = torch.tensor([[0.5, 0.5, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5]], dtype=torch.float32).to(device) # Corrigi o shape_sq para 0.0 para ser mais genérico ao dummy, ou mantive como era no original: 1.0 para quadrado
    else:
        detected_objects_features = []
        img_width, img_height = original_image.size

        # Mapeamento de classes YOLOv5 (COCO dataset) para 'formas' ClevrSimplified.
        # Esta é uma simplificação, pois YOLOv5 não detecta formas geométricas abstratas diretamente.
        # As classes de ClevrSimplified são: IsCircle(0), IsSquare(1), IsCylinder(2), IsCone(3), IsTriangle(4).
        # Exemplos de mapeamento para algumas classes comuns do COCO:
        yolo_class_to_clevr_shape = {
            2: 1, # 'car' -> IsSquare
            3: 1, # 'motorcycle' -> IsSquare (simplificação)
            7: 1, # 'truck' -> IsSquare
            5: 0, # 'bus' -> IsCircle (simplificação, para ter variedade)
            0: 2, # 'person' -> IsCylinder (simplificação)
            14: 3, # 'bird' -> IsCone (simplificação)
            # Adicione mais mapeamentos conforme a necessidade e a interpretação.
        }

        for *xyxy, conf, cls_id in filtered_detections:
            bbox = [val.item() for val in xyxy] # Extrai coordenadas da caixa delimitadora.
            xmin, ymin, xmax, ymax = bbox

            # a. Posição (centro da caixa delimitadora, normalizada para 0-1).
            center_x = (xmin + xmax) / 2 / img_width
            center_y = (ymin + ymax) / 2 / img_height
            pos_features = [center_x, center_y]

            # b. Cor (média RGB da caixa delimitadora, simplificado para 3 componentes one-hot-like).
            cropped_img_pil = original_image.crop((xmin, ymin, xmax, ymax)) # Recorta a área do objeto.
            cropped_img_np = np.array(cropped_img_pil) # Converte para array NumPy.

            if cropped_img_np.size == 0: # Caso a área recortada seja vazia.
                avg_rgb = [0.0, 0.0, 0.0]
            else:
                avg_rgb = np.mean(cropped_img_np, axis=(0, 1)) / 255.0 # Calcula a média RGB e normaliza para 0-1.

            # Simplificação para um formato one-hot-like de 3 cores (R, G, B).
            color_features = [0.0, 0.0, 0.0]
            if avg_rgb[0] > avg_rgb[1] and avg_rgb[0] > avg_rgb[2]:
                color_features[0] = 1.0 # Vermelho dominante.
            elif avg_rgb[1] > avg_rgb[0] and avg_rgb[1] > avg_rgb[2]:
                color_features[1] = 1.0 # Verde dominante.
            elif avg_rgb[2] > avg_rgb[0] and avg_rgb[2] > avg_rgb[1]:
                color_features[2] = 1.0 # Azul dominante.
            else:
                color_features[0] = 1.0 # Padrão para Vermelho se não houver dominância clara.

            # c. Forma (do rótulo de classe do YOLOv5, mapeado para Clevr).
            shape_one_hot = [0.0] * 5 # [IsCircle, IsSquare, IsCylinder, IsCone, IsTriangle]
            clevr_shape_idx = yolo_class_to_clevr_shape.get(int(cls_id.item()), -1) # Obtém o índice da forma Clevr.
            if 0 <= clevr_shape_idx < 5:
                shape_one_hot[clevr_shape_idx] = 1.0
            else:
                shape_one_hot[1] = 1.0 # Padrão para IsSquare se a classe YOLO não for mapeada.

            # d. Tamanho (área da caixa delimitadora, normalizada e mapeada para 'is_large' ou 'is_small').
            area = (xmax - xmin) * (ymax - ymin)
            normalized_area = area / (img_width * img_height) # Normaliza a área pela área total da imagem.
            is_large_feature = 1.0 if normalized_area > 0.05 else 0.0 # Threshold arbitrário para definir 'grande' ou 'pequeno'.
            size_features = [is_large_feature]

            # 4. Concatena todas as características para formar o vetor do objeto.
            object_vec = pos_features + color_features + shape_one_hot + size_features
            detected_objects_features.append(object_vec)

        # Converte a lista de vetores de características para um tensor PyTorch.
        detected_objects_features = torch.tensor(detected_objects_features, dtype=torch.float32).to(device)

    print(f"Características extraídas para {detected_objects_features.shape[0]} objetos.")
    print(f"Formato do tensor de características dos objetos: {detected_objects_features.shape}")

Carregando modelo YOLOv5 no dispositivo: cpu


/usr/local/lib/python3.12/dist-packages/torch/hub.py:335: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-12-15 Python-3.12.12 torch-2.9.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 99.5MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Modelo YOLOv5 carregado com sucesso.
Erro: A imagem original não foi carregada. Por favor, execute new_cell_5 primeiro.
Nenhum objeto detectado, usando objeto dummy como fallback.


## 4. Visualizar Objetos Detectados e Suas Características

### Subtask:
Visualizar a imagem com os objetos detectados (caixas delimitadoras) e exibir suas características extraídas (posição, cor, forma, tamanho).

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Usa a imagem original já carregada na célula `new_cell_5` ou `new_cell_7`.
# `original_image` deve estar disponível globalmente.
if 'original_image' not in locals() or original_image is None:
    print("Erro: original_image não encontrada. Por favor, execute as células de carregamento de imagem primeiro.")
    # Fallback para evitar erro, mas o usuário deve reexecutar a célula new_cell_5
    original_image = Image.new('RGB', (600, 400), color = 'white') # Cria uma imagem em branco
    img_width, img_height = original_image.size
else:
    img_width, img_height = original_image.size

# Cria uma figura e um eixo para exibir a imagem
fig, ax = plt.subplots(1, figsize=(12, 12))
ax.imshow(original_image)
ax.set_title("Objetos Detectados e Características")
ax.axis('off') # Remove os eixos

print("--- Detalhes dos Objetos ---")

# Itera sobre os objetos em detected_objects_features para visualização
# `detected_objects_features` é o tensor que contém [pos_x, pos_y, color_r, color_g, color_b, shape_circ, shape_sq, shape_cyl, shape_cone, shape_tri, size]
# Lembre-se que `filtered_detections` (do YOLO) é a fonte original das bboxes para objetos reais.

if 'filtered_detections' in locals() and len(filtered_detections) > 0: # Caso objetos reais tenham sido detectados
    for i, (*xyxy, conf, cls_id) in enumerate(filtered_detections):
        bbox = [val.item() for val in xyxy]  # xmin, ymin, xmax, ymax
        xmin, ymin, xmax, ymax = bbox
        width = xmax - xmin
        height = ymax - ymin

        # Desenha a caixa delimitadora
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

        # Obtém as características do objeto no formato LTN para exibição
        obj_feature_vec = detected_objects_features[i]

        # Posição
        pos_x_norm = obj_feature_vec[0].item()
        pos_y_norm = obj_feature_vec[1].item()
        center_x_pixel = pos_x_norm * img_width
        center_y_pixel = pos_y_norm * img_height

        # Cor (índices 2, 3, 4)
        colors_map = {0:'Vermelho', 1:'Verde', 2:'Azul'}
        color_idx = torch.argmax(obj_feature_vec[2:5]).item()
        inferred_color = colors_map.get(color_idx, 'Desconhecida')

        # Forma (índices 5, 6, 7, 8, 9)
        shapes_map = {0:'Círculo', 1:'Quadrado', 2:'Cilindro', 3:'Cone', 4:'Triângulo'}
        shape_idx = torch.argmax(obj_feature_vec[5:10]).item()
        inferred_shape = shapes_map.get(shape_idx, 'Desconhecida')

        # Tamanho (índice 10)
        inferred_size = "Grande" if obj_feature_vec[10].item() == 1.0 else "Pequeno"

        # Texto para o rótulo
        label_text = f"Objeto {i+1}:\nPos: ({pos_x_norm:.2f}, {pos_y_norm:.2f})\nCor: {inferred_color}\nForma: {inferred_shape}\nTamanho: {inferred_size}"
        ax.text(xmin, ymin - 10, label_text, color='lime', fontsize=9, bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))

        print(f"Objeto {i+1} (Original YOLO class: {model.names[int(cls_id.item())]}):")
        print(f"  - Posição (norm): ({pos_x_norm:.2f}, {pos_y_norm:.2f})")
        print(f"  - Cor inferida: {inferred_color}")
        print(f"  - Forma inferida: {inferred_shape}")
        print(f"  - Tamanho inferido: {inferred_size}")
        print(f"  - Confiança da detecção: {conf.item():.2f}")

else: # Caso apenas o objeto dummy esteja presente
    print("Nenhum objeto real detectado com confiança suficiente. Visualizando o objeto dummy.")
    obj_feature_vec = detected_objects_features[0] # Pega as características do dummy

    # Posição do dummy (centro da imagem)
    pos_x_norm = obj_feature_vec[0].item()
    pos_y_norm = obj_feature_vec[1].item()
    center_x_pixel = pos_x_norm * img_width
    center_y_pixel = pos_y_norm * img_height

    # Cor do dummy
    colors_map = {0:'Vermelho', 1:'Verde', 2:'Azul'}
    color_idx = torch.argmax(obj_feature_vec[2:5]).item()
    inferred_color = colors_map.get(color_idx, 'Desconhecida')

    # Forma do dummy
    shapes_map = {0:'Círculo', 1:'Quadrado', 2:'Cilindro', 3:'Cone', 4:'Triângulo'}
    shape_idx = torch.argmax(obj_feature_vec[5:10]).item()
    inferred_shape = shapes_map.get(shape_idx, 'Desconhecida')

    # Tamanho do dummy
    inferred_size = "Grande" if obj_feature_vec[10].item() == 1.0 else "Pequeno"

    # Desenha um ponto para representar o objeto dummy (sem bbox real)
    ax.plot(center_x_pixel, center_y_pixel, 'ro', markersize=15, label='Objeto Dummy')
    label_text = f"Objeto Dummy:\nPos: ({pos_x_norm:.2f}, {pos_y_norm:.2f})\nCor: {inferred_color}\nForma: {inferred_shape}\nTamanho: {inferred_size}"
    ax.text(center_x_pixel + 20, center_y_pixel - 20, label_text, color='red', fontsize=10, bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))
    ax.legend()

    print(f"Objeto Dummy:")
    print(f"  - Posição (norm): ({pos_x_norm:.2f}, {pos_y_norm:.2f})")
    print(f"  - Cor inferida: {inferred_color}")
    print(f"  - Forma inferida: {inferred_shape}")
    print(f"  - Tamanho inferido: {inferred_size}")
    print("  - Nota: Sem confiança de detecção real, este é um placeholder.")

plt.tight_layout()
plt.show()

Erro: original_image não encontrada. Por favor, execute as células de carregamento de imagem primeiro.
--- Detalhes dos Objetos ---
Nenhum objeto real detectado com confiança suficiente. Visualizando o objeto dummy.
Objeto Dummy:
  - Posição (norm): (0.50, 0.50)
  - Cor inferida: Verde
  - Forma inferida: Quadrado
  - Tamanho inferido: Pequeno
  - Nota: Sem confiança de detecção real, este é um placeholder.


## 5. Tentar Nova Imagem com Objetos Detectáveis

### Subtask:
Baixar uma nova imagem com objetos claros e visíveis para teste de detecção e atualizar a variável `image_path`.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Usa a imagem original já carregada na célula `new_cell_5` ou `new_cell_7`.
# `original_image` deve estar disponível globalmente.
if 'original_image' not in locals() or original_image is None:
    print("Erro: original_image não encontrada. Por favor, execute as células de carregamento de imagem primeiro.")
    # Fallback para evitar erro, mas o usuário deve reexecutar a célula new_cell_5
    original_image = Image.new('RGB', (600, 400), color = 'white') # Cria uma imagem em branco
    img_width, img_height = original_image.size
else:
    img_width, img_height = original_image.size

# Cria uma figura e um eixo para exibir a imagem
fig, ax = plt.subplots(1, figsize=(12, 12))
ax.imshow(original_image)
ax.set_title("Objetos Detectados e Características")
ax.axis('off') # Remove os eixos

print("--- Detalhes dos Objetos ---")

# Itera sobre os objetos em detected_objects_features para visualização
# `detected_objects_features` é o tensor que contém [pos_x, pos_y, color_r, color_g, color_b, shape_circ, shape_sq, shape_cyl, shape_cone, shape_tri, size]
# Lembre-se que `filtered_detections` (do YOLO) é a fonte original das bboxes para objetos reais.

if 'filtered_detections' in locals() and len(filtered_detections) > 0: # Caso objetos reais tenham sido detectados
    for i, (*xyxy, conf, cls_id) in enumerate(filtered_detections):
        bbox = [val.item() for val in xyxy]  # xmin, ymin, xmax, ymax
        xmin, ymin, xmax, ymax = bbox
        width = xmax - xmin
        height = ymax - ymin

        # Desenha a caixa delimitadora
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

        # Obtém as características do objeto no formato LTN para exibição
        obj_feature_vec = detected_objects_features[i]

        # Posição
        pos_x_norm = obj_feature_vec[0].item()
        pos_y_norm = obj_feature_vec[1].item()
        center_x_pixel = pos_x_norm * img_width
        center_y_pixel = pos_y_norm * img_height

        # Cor (índices 2, 3, 4)
        colors_map = {0:'Vermelho', 1:'Verde', 2:'Azul'}
        color_idx = torch.argmax(obj_feature_vec[2:5]).item()
        inferred_color = colors_map.get(color_idx, 'Desconhecida')

        # Forma (índices 5, 6, 7, 8, 9)
        shapes_map = {0:'Círculo', 1:'Quadrado', 2:'Cilindro', 3:'Cone', 4:'Triângulo'}
        shape_idx = torch.argmax(obj_feature_vec[5:10]).item()
        inferred_shape = shapes_map.get(shape_idx, 'Desconhecida')

        # Tamanho (índice 10)
        inferred_size = "Grande" if obj_feature_vec[10].item() == 1.0 else "Pequeno"

        # Texto para o rótulo
        label_text = f"Objeto {i+1}:\nPos: ({pos_x_norm:.2f}, {pos_y_norm:.2f})\nCor: {inferred_color}\nForma: {inferred_shape}\nTamanho: {inferred_size}"
        ax.text(xmin, ymin - 10, label_text, color='lime', fontsize=9, bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))

        print(f"Objeto {i+1} (Original YOLO class: {model.names[int(cls_id.item())]}):")
        print(f"  - Posição (norm): ({pos_x_norm:.2f}, {pos_y_norm:.2f})")
        print(f"  - Cor inferida: {inferred_color}")
        print(f"  - Forma inferida: {inferred_shape}")
        print(f"  - Tamanho inferido: {inferred_size}")
        print(f"  - Confiança da detecção: {conf.item():.2f}")

else: # Caso apenas o objeto dummy esteja presente
    print("Nenhum objeto real detectado com confiança suficiente. Visualizando o objeto dummy.")
    obj_feature_vec = detected_objects_features[0] # Pega as características do dummy

    # Posição do dummy (centro da imagem)
    pos_x_norm = obj_feature_vec[0].item()
    pos_y_norm = obj_feature_vec[1].item()
    center_x_pixel = pos_x_norm * img_width
    center_y_pixel = pos_y_norm * img_height

    # Cor do dummy
    colors_map = {0:'Vermelho', 1:'Verde', 2:'Azul'}
    color_idx = torch.argmax(obj_feature_vec[2:5]).item()
    inferred_color = colors_map.get(color_idx, 'Desconhecida')

    # Forma do dummy
    shapes_map = {0:'Círculo', 1:'Quadrado', 2:'Cilindro', 3:'Cone', 4:'Triângulo'}
    shape_idx = torch.argmax(obj_feature_vec[5:10]).item()
    inferred_shape = shapes_map.get(shape_idx, 'Desconhecida')

    # Tamanho do dummy
    inferred_size = "Grande" if obj_feature_vec[10].item() == 1.0 else "Pequeno"

    # Desenha um ponto para representar o objeto dummy (sem bbox real)
    ax.plot(center_x_pixel, center_y_pixel, 'ro', markersize=15, label='Objeto Dummy')
    label_text = f"Objeto Dummy:\nPos: ({pos_x_norm:.2f}, {pos_y_norm:.2f})\nCor: {inferred_color}\nForma: {inferred_shape}\nTamanho: {inferred_size}"
    ax.text(center_x_pixel + 20, center_y_pixel - 20, label_text, color='red', fontsize=10, bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))
    ax.legend()

    print(f"Objeto Dummy:")
    print(f"  - Posição (norm): ({pos_x_norm:.2f}, {pos_y_norm:.2f})")
    print(f"  - Cor inferida: {inferred_color}")
    print(f"  - Forma inferida: {inferred_shape}")
    print("  - Tamanho inferido: {inferred_size}")
    print("  - Nota: Sem confiança de detecção real, este é um placeholder.")

plt.tight_layout()
plt.show()

--- Detalhes dos Objetos ---
Nenhum objeto real detectado com confiança suficiente. Visualizando o objeto dummy.
Objeto Dummy:
  - Posição (norm): (0.50, 0.50)
  - Cor inferida: Verde
  - Forma inferida: Quadrado
  - Tamanho inferido: {inferred_size}
  - Nota: Sem confiança de detecção real, este é um placeholder.


In [ ]:
# --- Define a confiança mínima para detecção de objetos ---
# Este valor é usado na célula `new_cell_7` para filtrar detecções do YOLOv5.
confidence_threshold = 0.5
print(f"Confiança mínima para detecção de objetos definida como: {confidence_threshold}")

Confiança mínima para detecção de objetos definida como: 0.5


In [ ]:
import torch

# --- CLASSE ImageObjectDataset para integração de dados da imagem ---
# Esta classe é uma adaptação da ClevrSimplified, mas em vez de gerar dados sintéticos,
# ela aceita um tensor de características de objetos já detectados na imagem.
class ImageObjectDataset:
    def __init__(self, detected_features_tensor):
        # Armazena o tensor de características e o move para o dispositivo configurado (CPU/GPU).
        self._data = detected_features_tensor.to(device);
        # Define o número de amostras com base na primeira dimensão do tensor.
        self.num_samples = self.data.shape[0];
        print(f"ImageObjectDataset inicializado com {self.num_samples} objetos.");
        print(f"Formato dos dados: {self.data.shape}");

    # Propriedade 'data' para compatibilidade com o framework LTN, permitindo acesso aos dados.
    @property
    def data(self):
        return self._data;

    # Setter para a propriedade 'data', embora não seja usado neste contexto, é bom manter.
    @data.setter
    def data(self, value):
        self._data = value;


# Verifica se a variável `detected_objects_features` está disponível e contém dados.
# Esta variável deve ter sido gerada na etapa anterior de extração de características.
if 'detected_objects_features' in locals() and detected_objects_features is not None and detected_objects_features.shape[0] > 0:
    print(f"'detected_objects_features' encontrado com {detected_objects_features.shape[0]} objetos.");
    # Instancia ImageObjectDataset com as características extraídas da imagem.
    ds_image = ImageObjectDataset(detected_objects_features);
    print(f"Dataset para imagem real criado com sucesso. Contém {ds_image.num_samples} objetos de formato {ds_image.data.shape}.");
else:
    print("'detected_objects_features' não foi encontrado ou está vazio. Criando um objeto dummy como fallback.");
    # Fallback: Se não houver objetos detectados, cria um objeto dummy para que o LTN possa operar sem erros.
    # Formato do dummy: [pos_x, pos_y, color_r, color_g, color_b, shape_circ, shape_sq, shape_cyl, shape_cone, shape_tri, size]
    # Exemplo: um objeto no centro, verde, quadrado, tamanho médio.
    dummy_features = torch.tensor([[0.5, 0.5, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.5]], dtype=torch.float32).to(device);
    ds_image = ImageObjectDataset(dummy_features);
    print(f"Criado dataset dummy como fallback. Contém {ds_image.num_samples} objeto de formato {ds_image.data.shape}.");

'detected_objects_features' encontrado com 1 objetos.
ImageObjectDataset inicializado com 1 objetos.
Formato dos dados: torch.Size([1, 11])
Dataset para imagem real criado com sucesso. Contém 1 objetos de formato torch.Size([1, 11]).


In [ ]:
# Baixa uma nova imagem para teste
!wget -O /content/multiple_objects.jpg https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg

# Atualiza o caminho da imagem para a nova imagem baixada
image_path = "/content/multiple_objects.jpg"

print(f"Caminho da imagem atualizado para: {image_path}")
print("Por favor, execute as células `new_cell_5`, `new_cell_7`, `new_cell_9`, `new_cell_3` e `new_cell_11` novamente para reexecutar o pipeline com a nova imagem.")

--2025-12-15 13:37:15--  https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168949 (165K) [image/jpeg]
Saving to: ‘/content/multiple_objects.jpg’

/content/multiple_o 100%[===================>] 164.99K  --.-KB/s    in 0.01s   

2025-12-15 13:37:15 (11.0 MB/s) - ‘/content/multiple_objects.jpg’ saved [168949/168949]

Caminho da imagem atualizado para: /content/multiple_objects.jpg
Por favor, execute as células `new_cell_5`, `new_cell_7`, `new_cell_9`, `new_cell_3` e `new_cell_11` novamente para reexecutar o pipeline com a nova imagem.


In [ ]:
# --- 3. CONFIGURAÇÃO E EXECUÇÃO DO EXPERIMENTO LTN ---
# Este bloco instancia os predicados LTN com os modelos neurais, define os operadores lógicos,
# configura o otimizador, executa o loop de treinamento e realiza a avaliação final.

# --- A. INSTANCIAÇÃO DOS PREDICADOS LTN ---
# Cada predicado LTN encapsula um modelo neural (FeatureModel ou RelationModel)
# que aprende a verdade de uma proposição. `.to(device)` move o modelo para a GPU/CPU.

# Predicados Unários (Propriedades de um único objeto):
# IsCircle, IsSquare, IsCylinder, IsCone, IsTriangle: Usam FeatureModel para checar a forma (índices 5-9).
IsCircle = ltn.Predicate(FeatureModel([5]).to(device))
IsSquare = ltn.Predicate(FeatureModel([6]).to(device))
IsCylinder = ltn.Predicate(FeatureModel([7]).to(device))
IsCone = ltn.Predicate(FeatureModel([8]).to(device))
IsTriangle = ltn.Predicate(FeatureModel([9]).to(device))

# IsGreen: Usa FeatureModel para checar a cor verde (índice 3).
IsGreen = ltn.Predicate(FeatureModel([3]).to(device))
# IsSmall: Usa FeatureModel para checar o tamanho (índice 10).
IsSmall = ltn.Predicate(FeatureModel([10]).to(device))

# Predicados Binários (Relações entre dois objetos):
# LeftOf, RightOf: Usam RelationModel para checar a relação no eixo X (índice 0).
LeftOf = ltn.Predicate(RelationModel(0).to(device))
RightOf = ltn.Predicate(RelationModel(0).to(device))
# Below, Above: Usam RelationModel para checar a relação no eixo Y (índice 1).
Below = ltn.Predicate(RelationModel(1).to(device))
Above = ltn.Predicate(RelationModel(1).to(device))

# CloseTo: Usa CloseToModel para verificar proximidade entre objetos.
CloseTo = ltn.Predicate(CloseToModel().to(device))
# SameSize: Usa SameSizeModel para verificar se objetos têm o mesmo tamanho.
SameSize = ltn.Predicate(SameSizeModel().to(device))

# --- OPERADORES LÓGICOS FUZZY (Conectivos e Quantificadores) ---
# LTN utiliza operadores lógicos fuzzy para calcular a 'verdade' de proposições complexas.
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
# Quantificadores (Forall para "para todo", Exists para "existe")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
# Agregador para combinar os valores de verdade de múltiplos axiomas/regras.
sat_agg = ltn.fuzzy_ops.AggregPMeanError(p=2)

# --- FUNÇÃO PRINCIPAL DE EXECUÇÃO DO EXPERIMENTO ---
# Esta função encapsula um único ciclo de treinamento e avaliação do LTN.
def run_experiment(run_id):
    print(f"\n--- INICIANDO EXECUÇÃO {run_id+1}/5 ---")

    # B. Geração de Dados e Variáveis LTN
    # Usa o dataset de objetos detectados na imagem (ds_image) ao invés de gerar dados sintéticos.
    global ds_image # Declara ds_image como global para acessar o objeto criado na célula 9.
    data = ds_image.data # Obtém o tensor de dados dos objetos da imagem.

    # Define variáveis LTN 'x', 'y' e 'z' sobre o conjunto de dados.
    # LTN usa essas variáveis para quantificar sobre os objetos nos axiomas.
    x = ltn.Variable("x", data)
    y = ltn.Variable("y", data)
    z = ltn.Variable("z", data)

    # C. Otimizador
    # Otimizador Adam para ajustar os pesos das redes neurais dentro dos predicados.
    # `list(Predicate.parameters())` coleta todos os parâmetros treináveis dos modelos neurais.
    optimizer = torch.optim.Adam(
        list(IsCircle.parameters()) + list(IsSquare.parameters()) +
        list(IsCylinder.parameters()) + list(IsCone.parameters()) + list(IsTriangle.parameters()) +
        list(IsSmall.parameters()) + list(IsGreen.parameters()) +
        list(LeftOf.parameters()) + list(RightOf.parameters()) +
        list(Below.parameters()) + list(Above.parameters()), lr=0.01
    )

    # D. Loop de Treino
    print("Treinando...")
    for epoch in range(300):
        optimizer.zero_grad() # Zera os gradientes acumulados de iterações anteriores.

        # --- AXIONAS LTN ---
        # Axiomas são proposições lógicas que o modelo deve aprender a satisfazer.
        # O objetivo do treinamento é maximizar o valor de verdade desses axiomas.

        # 1. Axiomas de Taxonomia:
        # sat_cov: Cobertura - Todo objeto deve ser uma das formas conhecidas (Círculo OU Quadrado OU Cilindro OU Cone OU Triângulo).
        sat_cov = Forall(x, Or(Or(Or(Or(IsCircle(x), IsSquare(x)), IsCylinder(x)), IsCone(x)), IsTriangle(x)))
        # sat_mut_1: Exclusão Mútua - Se é um Círculo, não pode ser um Quadrado.
        sat_mut_1 = Forall(x, Implies(IsCircle(x), Not(IsSquare(x))))
        # sat_mut_2: Exclusão Mútua - Se é um Quadrado, não pode ser um Cilindro.
        sat_mut_2 = Forall(x, Implies(IsSquare(x), Not(IsCylinder(x))))

        # 2. Axiomas Espaciais (Horizontal):
        # sat_lr_irref: Irreflexividade - Nenhum objeto pode estar à sua própria esquerda.
        sat_lr_irref = Forall(x, Not(LeftOf(x, x)))
        # sat_lr_inv: Inversão - Se X está à esquerda de Y, então Y está à direita de X.
        sat_lr_inv = Forall([x, y], Implies(LeftOf(x, y), RightOf(y, x)))
        # sat_lr_trans: Transitividade - Se X está à esquerda de Y, e Y à esquerda de Z, então X está à esquerda de Z.
        sat_lr_trans = Forall([x,y,z], Implies(And(LeftOf(x,y), LeftOf(y,z)), LeftOf(x,z)))

        # 3. Axiomas Verticais:
        # sat_ud_inv: Inversão - Se X está abaixo de Y, então Y está acima de X.
        sat_ud_inv = Forall([x, y], Implies(Below(x, y), Above(y, x)))
        # sat_ud_trans: Transitividade - Se X está abaixo de Y, e Y abaixo de Z, então X está abaixo de Z.
        sat_ud_trans = Forall([x,y,z], Implies(And(Below(x,y), Below(y,z)), Below(x,z)))

        # --- ADIÇÃO 1: REGRA DE PROXIMIDADE (Tarefa 3.3) ---
        # Nova regra: "Se dois objetos são Triângulos e estão Próximos, devem ter o mesmo Tamanho"
        # Isso demonstra a capacidade do LTN de incorporar regras complexas.
        sat_prox = Forall([x,y], Implies(And(IsTriangle(x), And(IsTriangle(y), CloseTo(x,y))), SameSize(x,y)))

        # 4. Axiomas de Supervisão (Grounding):
        # Estes axiomas supervisionam diretamente os predicados unários com base nos dados brutos.
        # Criam máscaras para identificar objetos de cada forma no conjunto de dados.
        mask_circ = data[:, 5] == 1
        mask_sq = data[:, 6] == 1
        mask_cyl = data[:, 7] == 1
        mask_cone = data[:, 8] == 1
        mask_tri = data[:, 9] == 1

        # Para cada forma, cria um axioma que diz: "Para todo objeto de tipo X, ele É X".
        # O `if mask_X.any() else ltn.Constant(torch.tensor(1.))` evita erros se não houver objetos de uma certa forma.
        s_circ = Forall(ltn.Variable("xc", data[mask_circ]), IsCircle(ltn.Variable("xc", data[mask_circ]))) if mask_circ.any() else ltn.Constant(torch.tensor(1.))
        s_sq = Forall(ltn.Variable("xs", data[mask_sq]), IsSquare(ltn.Variable("xs", data[mask_sq]))) if mask_sq.any() else ltn.Constant(torch.tensor(1.))
        s_cyl = Forall(ltn.Variable("xcy", data[mask_cyl]), IsCylinder(ltn.Variable("xcy", data[mask_cyl]))) if mask_cyl.any() else ltn.Constant(torch.tensor(1.))
        s_cone = Forall(ltn.Variable("xco", data[mask_cone]), IsCone(ltn.Variable("xco", data[mask_cone]))) if mask_cone.any() else ltn.Constant(torch.tensor(1.))
        s_tri = Forall(ltn.Variable("xt", data[mask_tri]), IsTriangle(ltn.Variable("xt", data[mask_tri]))) if mask_tri.any() else ltn.Constant(torch.tensor(1.))

        # Agregação da Loss:
        # Combina todos os valores de verdade dos axiomas usando `sat_agg`.
        # O objetivo é maximizar essa verdade agregada.
        sat_total = sat_agg(torch.stack([
            sat_cov.value, sat_mut_1.value, sat_mut_2.value,
            sat_lr_irref.value, sat_lr_inv.value, sat_lr_trans.value,
            sat_ud_inv.value, sat_ud_trans.value,
            sat_prox.value,  # <--- Valor de verdade da nova regra de proximidade
            s_circ.value, s_sq.value, s_cyl.value, s_cone.value, s_tri.value
        ]))

        # A função de perda é (1 - sat_total), ou seja, minimizar a perda é maximizar a verdade.
        loss = 1.0 - sat_total
        loss.backward() # Calcula os gradientes.
        optimizer.step() # Atualiza os pesos dos modelos neurais com base nos gradientes.

    print(f"Loss Final Run {run_id+1}: {loss.item():.4f}")

    # --- 5. AVALIAÇÃO E CONSULTAS FINAIS ---
    # Após o treinamento, formulamos e avaliamos consultas para verificar o conhecimento aprendido.

    # q1: Existe um objeto pequeno (IsSmall) que esteja abaixo de um Cilindro (IsCylinder) E à esquerda de um Quadrado (IsSquare)?
    q1 = Exists(x, And(IsSmall(x), And(Exists(y, And(IsCylinder(y), Below(x,y))), Exists(z, And(IsSquare(z), LeftOf(x,z))))))
    # q2: Existem 3 objetos (x, y, z) onde x é um Cone verde (IsCone e IsGreen), y está à esquerda de x, e x está à esquerda de z?
    q2 = Exists([x,y,z], And(And(IsCone(x), IsGreen(x)), And(LeftOf(y,x), LeftOf(x,z))))

    # C. Cálculo de Métricas de Classificação (para formas):
    # Avalia a capacidade dos predicados de forma em classificar corretamente os objetos.
    # true_shapes será baseado nos dados da imagem, se objetos reais foram detectados.
    true_shapes = torch.argmax(data[:, 5:10], dim=1).cpu().numpy()

    p_s = []
    for pred in [IsCircle, IsSquare, IsCylinder, IsCone, IsTriangle]:
        p_s.append(pred.model(data).detach().cpu().numpy())
    pred_shapes = np.argmax(np.stack(p_s, axis=1).squeeze(axis=2), axis=1)

    acc = accuracy_score(true_shapes, pred_shapes)
    prec = precision_score(true_shapes, pred_shapes, average='macro', zero_division=0)
    rec = recall_score(true_shapes, pred_shapes, average='macro', zero_division=0)
    f1 = f1_score(true_shapes, pred_shapes, average='macro', zero_division=0)

    # --- ADIÇÃO 2: Retorno incluindo sat_prox (Necessário para a tabela de resultados) ---
    # Retorna os valores de verdade das consultas e da regra de proximidade,
    # além das métricas de classificação.
    return {
        "sat_q1": q1.value.item(),
        "sat_q2": q2.value.item(),
        "sat_regra_prox": sat_prox.value.item(), # <--- Valor de verdade da regra de proximidade
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }

# --- 6. EXECUÇÃO DOS 5 EXPERIMENTOS ---
# Executa a função `run_experiment` várias vezes para obter resultados estatisticamente robustos.
# Note que o treinamento será repetido 5 vezes sobre os *mesmos* dados da imagem.
results = []
for i in range(5):
    res = run_experiment(i)
    results.append(res)

# --- 7. RELATÓRIO FINAL ---
# Calcula e imprime a média e o desvio padrão dos resultados de todas as execuções.
print("\n=== RESULTADOS FINAIS (MÉDIA DE 5 EXECUÇÕES) ===")
keys = results[0].keys()
for k in keys:
    values = [r[k] for r in results]
    mean_val = np.mean(values)
    std_val = np.std(values)
    print(f"{k}: {mean_val:.4f} (+/- {std_val:.4f})")


--- INICIANDO EXECUÇÃO 1/5 ---
Treinando...
Loss Final Run 1: 0.0004

--- INICIANDO EXECUÇÃO 2/5 ---
Treinando...
Loss Final Run 2: 0.0002

--- INICIANDO EXECUÇÃO 3/5 ---
Treinando...
Loss Final Run 3: 0.0002

--- INICIANDO EXECUÇÃO 4/5 ---
Treinando...
Loss Final Run 4: 0.0002

--- INICIANDO EXECUÇÃO 5/5 ---
Treinando...
Loss Final Run 5: 0.0002

=== RESULTADOS FINAIS (MÉDIA DE 5 EXECUÇÕES) ===
sat_q1: 0.0002 (+/- 0.0000)
sat_q2: 0.0001 (+/- 0.0000)
sat_regra_prox: 0.9999 (+/- 0.0000)
accuracy: 1.0000 (+/- 0.0000)
precision: 1.0000 (+/- 0.0000)
recall: 1.0000 (+/- 0.0000)
f1: 1.0000 (+/- 0.0000)
